In [1]:
# импорт всех необходимых функций
from help_functions import *

## Задача 1

In [2]:
def f(x):
    return np.arccos(x)

# max производных arccos
max_derivs = [1, 1.17188, 1, 1.17188, 1] # – может ошибка тут?? но вроде нет

def get_table1():
    n = 4
    interp_point = 0.35
    # задаем функцию
    f = lambda x: np.arccos(x)
    # список теоритических границ
    bounds = []
    for i in range(n):
        x = np.array([0, 0.1, 0.2, 0.3, 0.5, 0.6])
        y = np.array([f(k) for k in x])
        x, y = nearest_points(x, y, interp_point, i + 1)
        bound = np.abs(np.prod(interp_point - np.array(x))) * max_derivs[i] / math.factorial(i + 1)
        bounds.append(bound)
    
    # список абсолютных погр мн-в разных степеней
    Rs = [] 
    for i in range(n):
        x = np.array([0, 0.1, 0.2, 0.3, 0.5, 0.6])
        y = np.array([f(k) for k in x])
        x, y = nearest_points(x, y, interp_point, i + 1)
        coefs = newton_coefs(x, y)
        # считаем значение многочлена в точке
        value = value_at_point(interp_point, coefs, x)
        R = np.abs(value - f(interp_point))
        # добавляем в список полученную абсолютную погрешность
        Rs.append(R)
    
    return pd.DataFrame({'R': Rs, 'граница': bounds})

In [3]:
# значения получаются странными (граница меньше R, но коэф-ты перепроверяла, максимумы производных вроде как тоже верные)
# не понятно
get_table1()

,R,граница
0,0.052878,0.050000
1,0.001211,0.004395
2,0.000318,0.000187
3,0.000046,0.000014


In [4]:
def get_table2():
    # по равностоящим узлам
    a = 0.5
    b = 1.5
    n = 4
    # шаг
    h = 0.1
    f = lambda x: np.cos(x)
    # дробим отрезок на части длиной h
    x = np.linspace(a, b, int((b - a) / h)  + 1)
    y = [f(z) for z in x]
    interp_point = 0.45
    t = (interp_point - a) / h

    table = np.zeros((n + 1, 6))
    for k in range(n + 1):
        ds_y0 = finite_differences(y, k) # считаем разности 
        dk_y0 = ds_y0[-1]
        nk = get_Nk(t, k)
        nk_delta = nk * dk_y0
        pk = get_Pk(ds_y0, t, k)

        # оценка погрешности
        m_kpo = np.abs(max_derivs[k])
        n_kpo = get_Nk(t, k + 1)
        th_R = np.abs(n_kpo) * m_kpo * h ** (k + 1)
        # записываем в таблицу
        table[k] = np.array([dk_y0, nk, nk_delta, pk, f(interp_point) - pk, th_R])
        
    index = ['∆k*y0', 'N_k(t)', 'N_k*∆k*y0',
            f'P_k({interp_point})', f'f({interp_point}) - P_k({interp_point})',
            f'|R_k({interp_point})| <=']
    return pd.DataFrame(table.T, index=index)

In [5]:
get_table2()

,0,1,2,3,4
∆k*y0,0.877583,-0.052247,-0.008246,0.000604,0.000076
N_k(t),1.000000,-0.500000,0.375000,-0.312500,0.273438
N_k*∆k*y0,0.877583,0.026123,-0.003092,-0.000189,0.000021
P_k(0.45),0.877583,0.903706,0.900614,0.900425,0.900446
f(0.45) - P_k(0.45),0.022865,-0.003259,-0.000167,0.000022,0.000002
|R_k(0.45)| <=,0.050000,0.004395,0.000313,0.000032,0.000002


## Задание 2

In [6]:
def f(x):
    return np.arccos(x)

x = np.array([0, 0.1, 0.2, 0.3, 0.5, 0.6])
y = np.array([f(i) for i in x])
interp_point = 0.35
x, y = nearest_points(x, y, interp_point, len(x)) # построим многочлен 5й степени

coefs = newton_coefs(x, y)
print(coefs)

[1.266103672779499, -1.0333473322506672, -0.20394425221279877, -0.2418314309950376, -0.23816127753504696, -0.213694653177382]


In [7]:
x_var = sp.symbols('x')
polynom = sympy_poly(x, y, x_var, coefs)

true_value = 0.75
true_point = np.cos(true_value)
polynom

-1.03334733225067*x - 0.213694653177382*(x - 0.6)*(x - 0.5)*(x - 0.3)*(x - 0.2)*(x - 0.1) - 0.238161277535047*(x - 0.5)*(x - 0.3)*(x - 0.2)*(x - 0.1) - 0.241831430995038*(x - 0.5)*(x - 0.3)*(x - 0.2) - 0.203944252212799*(x - 0.3)*(x - 0.2) + 1.5761078724547

In [8]:
root = newton_root(polynom - true_value, 0.5, 0.000001, 100)

решение нашлось после 3 итераций.


In [9]:
estimation = float(polynom.subs({x_var : root}))
pd.DataFrame([true_point, root, root - true_point, true_value, estimation],
             index=['настоящая точка', 'корень', 'ошибка', 'настоящее значение', 'приближение'], columns=[''])

,
настоящая точка,0.731689
корень,0.732664973336608
ошибка,0.000976104462787175
настоящее значение,0.75
приближение,0.75


## Задание 3

In [10]:
def f(x):
    return 1 / (1 + x**2)
# по корням мн-на
interp_point = 0.15
n = 7
x = [np.cos(np.pi * (2 * k + 8) / 22) for k in range(n)]
y = [f(i) for i in x]

x, y = nearest_points(x, y, interp_point, n) 
coefs = newton_coefs(x, y)
estimation = value_at_point(interp_point, coefs, x)

pd.DataFrame({'приближение': [estimation], 'Рунге': [f(interp_point)], 'ошибка': [np.abs(estimation - f(interp_point))]})

,приближение,Рунге,ошибка
0,0.977887,0.977995,0.000108


In [11]:
# по равностоящим узлам
a = -1
b = 1
h = (b - a) / (n - 1)
x = np.linspace(a, b, n)
y = [f(i) for i in x]
t = interp_point / h

In [12]:
ds_y0 = finite_differences2(y, n - 1)
ds_y0 = [ds_y0[k][3 - k // 2] for k in range(n)] # получаем средний ряд
# считаем Nk
Nks = [1]
for i in range(1, n): Nks.append(Nks[-1] * (t - (-1)**i * (i // 2)) / i)
estimation = 0
for i, j in zip(ds_y0, Nks):
    estimation += i * j

In [13]:
pd.DataFrame({'приближение': [estimation],
              'Рунге': [f(interp_point)],
              'ошибка': [np.abs(estimation - f(interp_point))]})

,приближение,Рунге,ошибка
0,0.978246,0.977995,0.000251


## Задача 4

In [14]:
x = [-2, -2, -2, 1]
y = [24, 24, 24, 12]
dy = [39, 39, 39, None]
d2y = [-42, -42, -42, None]
# сортируем
inds = [m[1] for m in sorted(zip(x, range(len(x))))]
x = [x[i] for i in inds]
y = [y[i] for i in inds]
dy = [dy[i] for i in inds]
d2y = [d2y[i] for i in inds]
# производные 
derivatives = [y, dy, d2y]
# порядок
order = 5
# коэф-ты
coefs = divided_differences(x, y, order, derivatives)
print(coefs)

[24, 39.0, -21.0, 2.222222222222222]


In [15]:
# получаем многочлен по коэф-там
polynom = sympy_poly(x, y, x_var, coefs)
polynom

39.0*x + 2.22222222222222*(x + 2)**3 - 21.0*(x + 2)**2 + 102.0

In [16]:
# проверка
print(polynom.subs({x_var: -2}), '- значение в -2')
print(polynom.subs({x_var: 1}), '- значение в 1')
print(diff(polynom).subs({x_var: -2}), '- первая производная в -2')
print(diff(diff(polynom)).subs({x_var: -2}), '- вторая производная в -2')

24.0000000000000 - значение в -2
12.0000000000000 - значение в 1
39.0000000000000 - первая производная в -2
-42.0000000000000 - вторая производная в -2


## Задача 6

## 1

wolfram: 
$ \int_{0}^{1}\frac{\cos(x)}{\sqrt{x}} dx = 1.80905\$

## 2

In [17]:
simpson = (1 - 0) * np.cos(0.5)/np.sqrt(0.5)
simpson

1.2410891611274912

## 3

In [18]:
#узлы
nodes = np.array([1/4, 3/4])
#моменты весовой функции
moment = np.array([2, 2/3, 2/5, 2/7])
#коэфициенты
A = []
A.append((moment[0]*nodes[1]-moment[1])/(nodes[1]-nodes[0]))
A.append((moment[1]-moment[0]*nodes[0])/(nodes[1]-nodes[0]))
#приближенное значение интеграла
quadrature = 0
for i in range(2):
    quadrature += A[i]*np.cos(nodes[i])
quadrature

1.8587503258090148

## 4

In [19]:
#моменты уже посчитаны в пункте 3
#ищем коэфициенты многочлена
M1 = []
v1 =[]
for i in range(2):
    M1.append([moment[i], moment[i+1]]) 
    v1.append(-moment[i+2])
a0 = 3/35
a1 = -6/7
#находим корни -узлы
w = x_var ** 2 + a1 * x_var + a0
root = sp.solve(w)
#
A = []
A.append((moment[0]*root[1]-moment[1])/(root[1]-root[0]))
A.append((moment[1]-moment[0]*root[0])/(root[1]-root[0]))
quadrature = 0
for i in range(2):
    quadrature += A[i]*np.cos(float(root[i]))
print(quadrature)

1.80861639537771
